# Exercise - Treasury Arbitrage

#### Notation Commands

$$\newcommand{\Black}{\mathcal{B}}
\newcommand{\Blackcall}{\Black_{\mathrm{call}}}
\newcommand{\Blackput}{\Black_{\mathrm{put}}}
\newcommand{\EcondS}{\hat{S}_{\mathrm{conditional}}}
\newcommand{\Efwd}{\mathbb{E}^{T}}
\newcommand{\Ern}{\mathbb{E}^{\mathbb{Q}}}
\newcommand{\Tfwd}{T_{\mathrm{fwd}}}
\newcommand{\Tunder}{T_{\mathrm{bond}}}
\newcommand{\accint}{A}
\newcommand{\carry}{\widetilde{\cpn}}
\newcommand{\cashflow}{C}
\newcommand{\convert}{\phi}
\newcommand{\cpn}{c}
\newcommand{\ctd}{\mathrm{CTD}}
\newcommand{\disc}{Z}
\newcommand{\done}{d_{1}}
\newcommand{\dt}{\Delta t}
\newcommand{\dtwo}{d_{2}}
\newcommand{\flatvol}{\sigma_{\mathrm{flat}}}
\newcommand{\flatvolT}{\sigma_{\mathrm{flat},T}}
\newcommand{\float}{\mathrm{flt}}
\newcommand{\freq}{m}
\newcommand{\futprice}{\mathcal{F}(t,T)}
\newcommand{\futpriceDT}{\mathcal{F}(t+h,T)}
\newcommand{\futpriceT}{\mathcal{F}(T,T)}
\newcommand{\futrate}{\mathscr{f}}
\newcommand{\fwdprice}{F(t,T)}
\newcommand{\fwdpriceDT}{F(t+h,T)}
\newcommand{\fwdpriceT}{F(T,T)}
\newcommand{\fwdrate}{f}
\newcommand{\fwdvol}{\sigma_{\mathrm{fwd}}}
\newcommand{\fwdvolTi}{\sigma_{\mathrm{fwd},T_i}}
\newcommand{\grossbasis}{B}
\newcommand{\hedge}{\Delta}
\newcommand{\ivol}{\sigma_{\mathrm{imp}}}
\newcommand{\logprice}{p}
\newcommand{\logyield}{y}
\newcommand{\mat}{(n)}
\newcommand{\nargcond}{d_{1}}
\newcommand{\nargexer}{d_{2}}
\newcommand{\netbasis}{\tilde{\grossbasis}}
\newcommand{\normcdf}{\mathcal{N}}
\newcommand{\notional}{K}
\newcommand{\pfwd}{P_{\mathrm{fwd}}}
\newcommand{\pnl}{\Pi}
\newcommand{\price}{P}
\newcommand{\probexer}{\hat{\mathcal{P}}_{\mathrm{exercise}}}
\newcommand{\pvstrike}{K^*}
\newcommand{\refrate}{r^{\mathrm{ref}}}
\newcommand{\rrepo}{r^{\mathrm{repo}}}
\newcommand{\spotrate}{r}
\newcommand{\spread}{s}
\newcommand{\strike}{K}
\newcommand{\swap}{\mathrm{sw}}
\newcommand{\swaprate}{\cpn_{\swap}}
\newcommand{\tbond}{\mathrm{fix}}
\newcommand{\ttm}{\tau}
\newcommand{\value}{V}
\newcommand{\vega}{\nu}
\newcommand{\years}{\tau}
\newcommand{\yearsACT}{\tau_{\mathrm{act/360}}}
\newcommand{\yield}{Y}$$

# 1. Treasury Arbitrage

Consider the following market data as of `Dec 29, 2023`.

The table below shows two Treasury securities, a T-note and a T-bond. They mature on the same date.

In [16]:
import pandas as pd
from pandas import IndexSlice
import numpy as np

summary = pd.DataFrame(index=[],columns = [207391,204095],dtype=float)
summary.loc['issue date'] = ['2019-08-15','1999-08-15']
summary.loc['maturity date'] = ['2029-08-15','2029-08-15']
summary.loc['coupon rate'] = [.01625, .06125]
summary.loc['clean price'] = [89.03125,111.0391]
summary.loc['accrued interest'] = [.6005, 2.2636]
summary.loc['ytm'] = [.037677, .038784]

summary.style.format("{:.2%}", subset=IndexSlice[["ytm","coupon rate"], :]).format("{:.2f}", subset=IndexSlice[["accrued interest","clean price"], :])

,207391,204095
issue date,2019-08-15,1999-08-15
maturity date,2029-08-15,2029-08-15
coupon rate,1.62%,6.12%
clean price,89.03,111.04
accrued interest,0.60,2.26
ytm,3.77%,3.88%


As usual, the quotes are per $100 face value.

### 1.1.

Which bond would you go long, and which bond would you short?

Explain your reasoning.

---------

I would long the 3.88% and short the 3.77% bonds. Due to the discrepancy in YTMs for the same maturity we can do an arb.

### 1.2.

Explain how you would finance the trade? Be specific in explaining how you would raise the cash for the long position and how you would achieve the short position.

-----

The trade would be financed using REPO as discussed in class + an extra haircut. Short can be achieved by a reverse REPO.


### 1.3. 

What are the risks of this trade? Is it an arbitrage in the short-term or in the long-term? Explain.

-------

- Repo risk -> rising repo cost or lack of liqidity
- Further divergence -> the YTM spread can further increase and force a margin call
- Convexity and volatility risk -> can force the trade to become unprofitable

### 1.4.

Suppose that on `2024-02-15`, immediately after the coupon is paid out, we observe the prices are `87` and `113`, respectively.
* Approximate time-to-maturity as 5.5 years.
* Note that the coupon was just paid, so there is no accrued interest.

Calculate the new YTMs for the two bonds.

----------



In [18]:
new_prices = {207391: 87.0, 204095: 113.0}
ttm_years = 5.5
freq = 2  
N = int(round(ttm_years * freq)) 

def bond_price_from_ytm(y, coupon_rate, N, face=100.0, freq=2):
    c = coupon_rate * face / freq
    r = y / freq
    disc = (1.0 + r) ** np.arange(1, N + 1)
    pv_coupons = np.sum(c / disc)
    pv_face = face / ((1.0 + r) ** N)
    return pv_coupons + pv_face

def solve_ytm_bisection(price, coupon_rate, N, face=100.0, freq=2,
                        y_low=1e-8, y_high=1.0, tol=1e-12, max_iter=200):

    def f(y):
        return bond_price_from_ytm(y, coupon_rate, N, face=face, freq=freq) - price

    lo, hi = y_low, y_high
    f_lo, f_hi = f(lo), f(hi)

    while f_lo * f_hi > 0:
        hi *= 2
        if hi > 10: 
            raise ValueError("Could not bracket the YTM root. Check inputs.")
        f_hi = f(hi)

    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        f_mid = f(mid)

        if abs(f_mid) < tol or (hi - lo) < tol:
            return mid

        if f_lo * f_mid > 0:
            lo, f_lo = mid, f_mid
        else:
            hi, f_hi = mid, f_mid

    return 0.5 * (lo + hi)

new_ytms = {}
for cusip in summary.columns:
    coupon_rate = float(summary.loc["coupon rate", cusip])
    price = float(new_prices[cusip])  # no accrued interest, so dirty = clean
    ytm = solve_ytm_bisection(price=price, coupon_rate=coupon_rate, N=N, face=100.0, freq=freq)
    new_ytms[cusip] = ytm

results = pd.DataFrame(
    {
        "coupon_rate": [summary.loc["coupon rate", c] for c in summary.columns],
        "new_price": [new_prices[c] for c in summary.columns],
        "TTM_years": [ttm_years for _ in summary.columns],
        "N_periods": [N for _ in summary.columns],
        "new_ytm": [new_ytms[c] for c in summary.columns],
    },
    index=summary.columns
)

results["coupon_rate"] = (results["coupon_rate"] * 100).round(3)
results["new_ytm_pct"] = (results["new_ytm"] * 100).round(4)

results["coupon_rate"]
results["new_ytm_pct"]

207391    4.3047
204095    3.5056
Name: new_ytm_pct, dtype: float64

### 1.5.

Suppose that on `2024-08-15` we observe the YTMs are now 4.65% and 4.70%, respectively. 

Note that this observation is immediately **after** the bonds pay out coupons.

* Calculate the prices of both bonds.
* Compare this to the **dirty** prices inferred from the table above.

------

207391 : Price ≈ 86.642  |   dirty = 89.63175

204095 : Price ≈ 106.284   |   dirty = 113.3027

Both new prices are lower than the old dirty prices because yields are now higher than before



### 1.6.

Suppose that you have a position...
* long 1 unit of T-bond (ID=`204095`).
* short 1 unit of T-note (ID=`207391`).

(Note that we're not balancing the dollars or risk. Just assume long-short one unit of each.)


Calculate the value of this long-short position using the 
* prices in the given table
* prices from `1.5`

How did the value of the position change? 


--------

204095 : Dirty = 111.0391 + 2.2636 = 113.3027

207391 : Dirty = 89.03125 + 0.6005 = 89.63175

V spread ​ =113.3027 − 89.63175 = 23.67095

=======

from 

204095 price ≈ 106.284

207391 price ≈ 86.642

V_1.5 ​= 106.284 − 86.642 = 19.642

=======

V_delt = 19.642 - 23.67095 = −4.028

Since the bond yield decreased.

***